In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
**Build Model** ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to train our model on a hardware accelerator like the GPU or MPS,
if available. Let's check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)
or [torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are available, otherwise we use the CPU.



In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Define the Class
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred} \n {X}")

Predicted class: tensor([9]) 
 tensor([[[3.9802e-01, 3.5150e-01, 2.4817e-01, 7.5466e-01, 5.4210e-01,
          7.9902e-01, 8.1513e-01, 9.9671e-01, 2.7973e-01, 8.7392e-01,
          7.1121e-01, 8.4089e-01, 5.7831e-01, 5.7237e-02, 2.2994e-01,
          4.7072e-01, 9.3501e-01, 2.6951e-01, 3.9196e-01, 3.3296e-01,
          9.8649e-01, 3.1162e-02, 6.0784e-01, 8.5882e-01, 5.6583e-01,
          8.2709e-01, 4.0122e-01, 9.7144e-01],
         [6.9258e-01, 9.9394e-01, 5.2918e-01, 6.9696e-01, 3.4551e-01,
          8.4556e-01, 6.2378e-01, 2.1127e-01, 5.6929e-01, 3.8001e-01,
          9.2920e-01, 2.3528e-01, 9.1875e-01, 8.3831e-01, 5.6708e-01,
          4.7983e-01, 4.3303e-01, 1.0929e-01, 7.9021e-01, 8.2096e-02,
          9.9493e-01, 9.8227e-01, 3.3417e-01, 9.6130e-01, 8.5678e-01,
          7.3834e-01, 5.8599e-01, 2.1394e-02],
         [9.1809e-01, 2.4801e-01, 4.8416e-02, 1.3035e-01, 3.8421e-01,
          2.6514e-01, 5.4354e-01, 6.1760e-01, 1.4484e-02, 9.3397e-02,
          4.0467e-01, 2.5237e-01, 8

--------------




## Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)

print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.




In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 1.6576e-01,  8.0559e-02, -1.0609e-01, -2.8955e-01,  2.3406e-01,
          9.0722e-02, -5.4006e-01,  1.3332e-01, -6.8582e-04, -7.1086e-01,
          2.2670e-01,  3.4781e-01,  3.6784e-01,  9.4104e-02, -4.4424e-03,
         -5.3927e-01, -1.8189e-01,  3.1705e-01,  2.2195e-01, -5.5640e-01],
        [ 7.2669e-01,  7.0936e-02, -2.9644e-01, -3.6701e-01,  3.5238e-01,
         -9.3453e-02, -8.8480e-01, -1.4652e-01,  1.4381e-01, -6.2109e-01,
          3.7545e-01, -7.3073e-03,  3.7699e-02,  3.4682e-01, -1.2071e-02,
         -8.5902e-01,  5.7812e-02,  4.3451e-01,  6.6886e-01, -7.9684e-01],
        [ 4.5013e-01, -9.1437e-02, -1.0453e-01, -1.6306e-01,  1.1860e-01,
         -2.0632e-01, -5.3787e-01, -3.3522e-02, -1.0103e-01, -6.7470e-01,
          1.0775e-01,  3.3737e-01,  2.8508e-01,  2.5667e-01,  3.0749e-02,
         -7.9390e-01, -6.2882e-02,  3.0614e-01, -1.2466e-01, -6.4856e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1658, 0.0806, 0.0000, 0.0000, 0.2341,

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1008, -0.2949, -0.2448, -0.0973,  0.0016, -0.2144, -0.0620,  0.0790,
         -0.2253, -0.0201],
        [-0.1245, -0.3494, -0.1992,  0.0033,  0.1088, -0.2210, -0.1391,  0.0316,
         -0.2295,  0.0586],
        [-0.1383, -0.3502, -0.2712, -0.0388,  0.1459, -0.3403, -0.1358,  0.0519,
         -0.2875, -0.0030]], grad_fn=<AddmmBackward0>)


### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(pred_probab)

tensor([[0.1010, 0.0832, 0.0875, 0.1014, 0.1119, 0.0902, 0.1050, 0.1209, 0.0892,
         0.1095],
        [0.0972, 0.0776, 0.0902, 0.1104, 0.1227, 0.0882, 0.0958, 0.1136, 0.0875,
         0.1167],
        [0.0985, 0.0797, 0.0862, 0.1088, 0.1309, 0.0805, 0.0987, 0.1191, 0.0848,
         0.1128]], grad_fn=<SoftmaxBackward0>)


## Model Parameters
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0249,  0.0056,  0.0351,  ...,  0.0321, -0.0303, -0.0231],
        [ 0.0076, -0.0357,  0.0206,  ...,  0.0250, -0.0021,  0.0319]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0077, -0.0321], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0241,  0.0071, -0.0014,  ...,  0.0020, -0.0113,  0.0322],
        [-0.0359, -0.0360, -0.0011,  ...,  0.0024,  0.0175, -0.0115]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

--------------




## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

